# 🎮 Battle City + DreamerV3

Тренировка world-model агента DreamerV3 для Battle City на NES.

**Runtime:** GPU (T4)

In [ ]:
#@title 1. Установка зависимостей
!pip install nes-py gymnasium jax[cuda12] jaxlib ninjax elements portal ruamel.yaml cloudpickle chex -q
print("✅ Зависимости установлены")

In [ ]:
#@title 2. Патч nes-py (NumPy 2.0)
rom_path = "/usr/local/lib/python3.12/dist-packages/nes_py/_rom.py"
with open(rom_path, 'r') as f:
    content = f.read()
content = content.replace(
    "return self.prg_rom_start + self.prg_rom_size * 2**10",
    "return int(self.prg_rom_start) + int(self.prg_rom_size) * 1024"
)
content = content.replace(
    "return self.chr_rom_start + self.chr_rom_size * 2**10",
    "return int(self.chr_rom_start) + int(self.chr_rom_size) * 1024"
)
with open(rom_path, 'w') as f:
    f.write(content)
print("✅ nes-py патч применён")

In [ ]:
#@title 3. Клонирование и патч JAX API
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /content/battle-city
!git clone https://github.com/Nikolay1221/battle-city-dreamer.git /content/battle-city

# Патч transform.py
transform_path = "/content/battle-city/embodied/jax/transform.py"
with open(transform_path, 'r') as f:
    content = f.read()
content = content.replace(
    "fn = jax.jit(fn, arg_shardings, params_sharding, static_argnums, None)",
    "fn = jax.jit(fn, in_shardings=arg_shardings, out_shardings=params_sharding, static_argnums=static_argnums)"
)
content = content.replace(
    "fn = jax.jit(fn, in_shardings, out_shardings, static_argnums, None, donate)",
    "fn = jax.jit(fn, in_shardings=in_shardings, out_shardings=out_shardings, static_argnums=static_argnums, donate_argnums=donate)"
)
with open(transform_path, 'w') as f:
    f.write(content)
print("✅ transform.py патч")

# Патч agent.py
agent_path = "/content/battle-city/embodied/jax/agent.py"
with open(agent_path, 'r') as f:
    content = f.read()
content = content.replace(
    """self._split = jax.jit(
        lambda xs: jax.tree.map(lambda x: list(x), xs),
        internal.local_sharding(self.policy_sharded),
        internal.local_sharding(self.policy_mirrored))
    self._stack = jax.jit(
        lambda xs: jax.tree.map(
            jnp.stack, xs, is_leaf=lambda x: isinstance(x, list)),
        internal.local_sharding(self.policy_mirrored),
        internal.local_sharding(self.policy_sharded))""",
    """self._split = jax.jit(
        lambda xs: jax.tree.map(lambda x: list(x), xs),
        in_shardings=internal.local_sharding(self.policy_sharded),
        out_shardings=internal.local_sharding(self.policy_mirrored))
    self._stack = jax.jit(
        lambda xs: jax.tree.map(
            jnp.stack, xs, is_leaf=lambda x: isinstance(x, list)),
        in_shardings=internal.local_sharding(self.policy_mirrored),
        out_shardings=internal.local_sharding(self.policy_sharded))"""
)
with open(agent_path, 'w') as f:
    f.write(content)
print("✅ agent.py патч")

# Патч configs.yaml (размер 64x64)
configs_path = "/content/battle-city/dreamerv3/configs.yaml"
with open(configs_path, 'r') as f:
    content = f.read()
content = content.replace("size: [52, 52]", "size: [64, 64]")
with open(configs_path, 'w') as f:
    f.write(content)
print("✅ configs.yaml 64x64")

# Патч battlecity.py
bc_path = "/content/battle-city/embodied/envs/battlecity.py"
with open(bc_path, 'r') as f:
    content = f.read()
content = content.replace("size=(52, 52)", "size=(64, 64)")
# Убираем импорт nes_py_patch (патч уже применён выше)
content = content.replace("import nes_py_patch  # noqa: F401", "# nes_py_patch applied in notebook")
with open(bc_path, 'w') as f:
    f.write(content)
print("✅ battlecity.py 64x64")

print("\n🎉 Всё готово!")

In [ ]:
#@title 4. Проверка окружения
import os
os.chdir('/content/battle-city')

from battle_city_env import BattleCityEnv
env = BattleCityEnv(render_mode='rgb_array')
obs, _ = env.reset()
print(f"Obs shape: {obs.shape}")
env.close()
print("✅ BattleCityEnv работает!")

In [ ]:
#@title 5. Запуск тренировки 🚀
import os
os.chdir('/content/battle-city')

LOGDIR = "/content/drive/MyDrive/battlecity_dreamer/logs"
os.makedirs(LOGDIR, exist_ok=True)

!python dreamerv3/main.py \
    --task battlecity_stage0 \
    --configs battlecity size12m \
    --logdir {LOGDIR} \
    --run.steps 5000000 \
    --run.envs 8 \
    --jax.platform cuda

In [ ]:
#@title 6. TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/battlecity_dreamer/logs